## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [13]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [14]:
train, test = keras.datasets.cifar10.load_data()

In [15]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [16]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [17]:
from keras import regularizers

def build_mlp(input_shape, l1_ratio):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)

    num_neurons=[512, 256, 128]
    for i, n_units in enumerate(num_neurons):
      if i == 0:
          x = keras.layers.Dense(units=n_units, 
                                 activation="relu", 
                                 name="hidden_layer"+str(i+1), 
                                 kernel_regularizer=keras.regularizers.l1(l1_ratio))(input_layer)
      else:
          x = keras.layers.Dense(units=n_units, 
                                 activation="relu", 
                                 name="hidden_layer"+str(i+1),
                                 kernel_regularizer=keras.regularizers.l1(l1_ratio))(x)
    output_units=10
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)

    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    
    return model

In [18]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 30
BATCH_SIZE = 256
MOMENTUM = 0.95
L1_EXP = [1e-2, 1e-4, 1e-8, 1e-12]

In [19]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for regulizer_ratio in L1_EXP:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with Regulizer = %.6f" % (regulizer_ratio))
    model = build_mlp(input_shape=x_train.shape[1:], l1_ratio=regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l1-%s" % str(regulizer_ratio)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

W0717 09:34:37.260809 15588 deprecation_wrapper.py:119] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 09:34:37.360806 15588 deprecation_wrapper.py:119] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Experiment with Regulizer = 0.010000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


W0717 09:34:37.492809 15588 deprecation.py:323] From C:\Users\yjlai\Anaconda3\envs\keras35\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 7s 140us/step - loss: 198.6014 - acc: 0.2457 - val_loss: 41.0580 - val_acc: 0.2565
Epoch 2/30
50000/50000 [==============================] - 3s 58us/step - loss: 19.0663 - acc: 0.1175 - val_loss: 7.2794 - val_acc: 0.1000
Epoch 3/30
50000/50000 [==============================] - 3s 58us/step - loss: 4.1108 - acc: 0.0997 - val_loss: 2.6468 - val_acc: 0.1000
Epoch 4/30
50000/50000 [==============================] - 3s 58us/step - loss: 2.4860 - acc: 0.0982 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 5/30
50000/50000 [==============================] - 3s 58us/step - loss: 2.4626 - acc: 0.0964 - val_loss: 2.4624 - val_acc: 0.1000
Epoch 6/30
50000/50000 [==============================] - 3s 58us/step - loss: 2.4626 - acc: 0.0981 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 7/30
50000/50000 [==============================] - 3s 58us/step - loss: 2.4626 - acc: 0.1003 - val_loss: 2.4626 - va

50000/50000 [==============================] - 3s 58us/step - loss: 4.0220 - acc: 0.5504 - val_loss: 4.1393 - val_acc: 0.4880
Epoch 23/30
50000/50000 [==============================] - 3s 57us/step - loss: 3.9563 - acc: 0.5554 - val_loss: 4.1160 - val_acc: 0.4846
Epoch 24/30
50000/50000 [==============================] - 3s 58us/step - loss: 3.8928 - acc: 0.5619 - val_loss: 3.9792 - val_acc: 0.5165
Epoch 25/30
50000/50000 [==============================] - 3s 57us/step - loss: 3.8347 - acc: 0.5644 - val_loss: 3.9798 - val_acc: 0.5015
Epoch 26/30
50000/50000 [==============================] - 3s 56us/step - loss: 3.7735 - acc: 0.5674 - val_loss: 3.8681 - val_acc: 0.5195
Epoch 27/30
50000/50000 [==============================] - 3s 57us/step - loss: 3.7140 - acc: 0.5702 - val_loss: 3.8359 - val_acc: 0.5120
Epoch 28/30
50000/50000 [==============================] - 3s 57us/step - loss: 3.6607 - acc: 0.5731 - val_loss: 3.7664 - val_acc: 0.5190
Epoch 29/30
50000/50000 [=====================

Epoch 6/30
50000/50000 [==============================] - 3s 57us/step - loss: 1.5756 - acc: 0.4487 - val_loss: 1.5705 - val_acc: 0.4442
Epoch 7/30
50000/50000 [==============================] - 3s 57us/step - loss: 1.5434 - acc: 0.4604 - val_loss: 1.5467 - val_acc: 0.4603
Epoch 8/30
50000/50000 [==============================] - 3s 58us/step - loss: 1.5127 - acc: 0.4707 - val_loss: 1.5287 - val_acc: 0.4678
Epoch 9/30
50000/50000 [==============================] - 3s 58us/step - loss: 1.4874 - acc: 0.4801 - val_loss: 1.5075 - val_acc: 0.4662
Epoch 10/30
50000/50000 [==============================] - 3s 58us/step - loss: 1.4622 - acc: 0.4887 - val_loss: 1.4951 - val_acc: 0.4683
Epoch 11/30
50000/50000 [==============================] - 3s 57us/step - loss: 1.4381 - acc: 0.4971 - val_loss: 1.4705 - val_acc: 0.4856
Epoch 12/30
50000/50000 [==============================] - 3s 57us/step - loss: 1.4155 - acc: 0.5029 - val_loss: 1.4552 - val_acc: 0.4816
Epoch 13/30
50000/50000 [=============

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond+'train', color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond+'val', color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 10])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond+'train', color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond+'val', color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()